In [ ]:
# Import openAWSEM, open3SPN2 and other libraries
import open3SPN2
import ffAWSEM

import pandas
import numpy as np
import openmm

from functools import partial
import sys

# System creation using pandas

In [ ]:
# Fix the system (adds missing atoms)
fix=open3SPN2.fixPDB("1le5.pdb")

In [ ]:
# Create a table containing both the proteins and the DNA
complex_table=open3SPN2.pdb2table(fix)

# Create a single memory file
ffAWSEM.create_single_memory(fix)

In [ ]:
#Generate a coarse-grained model of the DNA molecules
dna_atoms=open3SPN2.DNA.CoarseGrain(complex_table)

# Generate a coarse-grained model of the Protein molecules
protein_atoms=ffAWSEM.Protein.CoarseGrain(complex_table)

In [ ]:
dna_atoms.chainID.unique(),protein_atoms.chainID.unique()

In [ ]:
# Select appropiate chains for DNA and protein
dna_atoms=dna_atoms[dna_atoms['chainID'].isin(['C','D'])]
protein_atoms=protein_atoms[protein_atoms['chainID'].isin(['A','B'])]


In [ ]:
# Get original dna_sequence
dna_seq=''.join(dna_atoms[(dna_atoms['name']=='S') & (dna_atoms['chainID'].isin(['C']))]['resname'].str[1:])

In [ ]:
# Pad the sequence with new bases
num_copies = 4
pad_sequence = 'GC' * 100
#pad_sequence = 'AT' * 100
#pad_sequence = 'GCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGC'

# Compute the length of each segment of the pad_sequence
segment_length = len(pad_sequence) // num_copies

# Check if it is possible to insert the specified number of copies of dna_seq into pad_sequence
if segment_length < len(dna_seq):
    raise RuntimeError(f"Cannot insert {num_copies} copies of dna_seq of length {len(dna_seq)} into pad_sequence of length {len(pad_sequence)}.\n\
                         Maximum allowed number of copies is {len(pad_sequence) // len(dna_seq)}.")

# Insert the sequence in equidistant sites with padding.
new_sequence = ''
binding_sites=[]
for i in range(num_copies):
    start = i * segment_length
    end = start + segment_length
    middle = start + (segment_length - len(dna_seq)) // 2
    new_sequence += pad_sequence[start:middle] + dna_seq + pad_sequence[middle+len(dna_seq):end]
    binding_sites+=[(middle,middle+len(dna_seq))]

# Complete the pad sequence at the end if the length of the new sequence is less than the pad sequence
if len(new_sequence) < len(pad_sequence):
    # Add the remaining pad_sequence to the end of the new sequence
    new_sequence += pad_sequence[len(new_sequence):]

print(new_sequence)
print(len(new_sequence))


In [ ]:
print(binding_sites[0])
print(dna_seq)
print(new_sequence[binding_sites[0][0]:binding_sites[0][1]])


In [ ]:
# Generate new DNA
dna_padded=open3SPN2.DNA.fromSequence(new_sequence,dna_type='B_curved').atoms

In [ ]:
# Calculate the translation and rotation of the protein to the DNA
import prody
bs=binding_sites[0]
proteins=[]
for bs in binding_sites:
    sel = (dna_padded['resSeq'].isin(range(1+bs[0],1+bs[1])) & (dna_padded['chainID']=='A') & ~((dna_padded['resSeq']==1+bs[0]) & (dna_padded['name']=='S'))) |\
          (dna_padded['resSeq'].isin(range(len(new_sequence)-bs[1]-1,len(new_sequence)-bs[0]-1)) & (dna_padded['chainID']=='B') & ~((dna_padded['resSeq']==len(new_sequence)-bs[1]-1) & (dna_padded['name']=='S')))
    len(dna_padded[sel]),len(dna_atoms)

    #Calculate the rotation and translation needed to superpose the coordinates
    coords1=dna_padded[sel][['x','y','z']].values
    coords_dna=dna_atoms[['x','y','z']].values
    coords_protein=protein_atoms[['x','y','z']].values

    transformation = prody.calcTransformation(coords_dna, coords1)
    rotation, translation = transformation.getRotation(), transformation.getTranslation()

    # Apply the transformation to coords2
    transformed_dna = np.dot(coords_dna, rotation.T) + translation
    transformed_protein = np.dot(coords_protein, rotation.T) + translation


    # Compute the RMSD before superposition
    diff_before = coords1 - coords_dna
    rmsd_before = np.sqrt(np.mean(np.sum(diff_before ** 2, axis=1)))

    # Compute the RMSD after superposition
    diff_after = coords1 - transformed_dna
    rmsd_after = np.sqrt(np.mean(np.sum(diff_after ** 2, axis=1)))

    print(f"RMSD before superposition: {rmsd_before:.3f}, RMSD after superposition: {rmsd_after:.3f}")
    dna_padded.loc[sel,['x','y','z']]=transformed_dna
    protein_temp = protein_atoms.copy()
    protein_temp[['x','y','z']]=transformed_protein
    proteins+=[protein_temp]

In [ ]:
# Calculate anchor indices
anchors = (dna_padded[(dna_padded['name']=='S') & (dna_padded['chainID'].isin(['A']))].iloc[0].name, 
           dna_padded[(dna_padded['name']=='S') & (dna_padded['chainID'].isin(['A']))].iloc[-1].name, 
           dna_padded[(dna_padded['name']=='S') & (dna_padded['chainID'].isin(['B']))].iloc[0].name, 
           dna_padded[(dna_padded['name']=='S') & (dna_padded['chainID'].isin(['B']))].iloc[-1].name)
anchors

In [ ]:
# Calculate anchor coordinates for vectors T1 and T2 and the extreme of the DNA
anchor_coords=np.array([dna_padded.loc[a][['x','y','z']].values for a in anchors])
T1=anchor_coords[[0,-1]]
T2=anchor_coords[[1,2]]
T1,T2

In [ ]:
#Calculate distance between sugars and helical vector
dS=(((T2[1]-T2[0])**2).sum()**.5+((T1[1]-T1[0])**2).sum()**.5)/2
M=T2.mean(axis=0)-T1.mean(axis=0)
dS,M

In [ ]:
# Calculate projection of the T1 and T2 vectors in the XY plane
v1=(T1[0]-T1[1])
v2=(T2[0]-T2[1])
v1[-1]=0
v2[-1]=0
v1=v1/(v1**2).sum()**.5
v2=v2/(v2**2).sum()**.5
v1,v2

In [ ]:
# Compute coordinates of motors to bias the twisting of the DNA
n_motors=5
alpha=np.linspace(0,1,n_motors+1)[:,np.newaxis]
centers=T1.mean(axis=0)+M[np.newaxis,:]*alpha
v=v2*alpha+v1*(1-alpha)
A1=centers+v*dS/2
A2=centers-v*dS/2
A=np.array([A1,A2])
A=A.transpose(1,0,2).reshape(2*(n_motors+1),-1)

In [ ]:
# Add topology information to the motors to bias the system
motors=pandas.DataFrame(A,columns=['x','y','z'])
motors['chainID']='A'
motors['resname']=['X','X']*(n_motors+1)
motors['name']=['X1','X2']*(n_motors+1)
motors['resSeq']=np.repeat(np.arange(n_motors+1)+1,2)
motors['serial']=motors.index
motors['recname']='ATOM'
motors['altLoc']=''
motors['element']='C'

In [ ]:
# Define a function for chain renaming
import string
import itertools
def chain_name_generator(format='pdb'):
    allowed_characters = string.ascii_uppercase + string.ascii_lowercase + '0123456789'
    r = 0
    while True:
        r += 1
        if r == 2 and format == 'pdb':
            warnings.warn('The number of chains has exceeded the maximum allowable in the pdb format')
            break
        for a in itertools.product(allowed_characters, repeat=r):
            yield ''.join(a)
    while True:
        yield 'A'


In [ ]:
# Merge the model (DNA + proteins + motors)
chainID = []
name_generator = chain_name_generator()
scene_list=[dna_padded]+proteins+[motors]
for scene in scene_list:
    if 'chainID' not in scene:
        chainID += [next(name_generator)]*len(scene)
    else:
        chains = list(scene['chainID'].unique())
        chains.sort()
        chain_replace = {chain: next(name_generator) for chain in chains}
        chainID += list(scene['chainID'].replace(chain_replace))
name_generator.close()
model = pandas.concat(scene_list)
model['chainID'] = chainID
model.index = np.arange(len(model))
model.serial = np.arange(len(model))+1
model['resName']=model['resname']
model=model.fillna('')
model

In [ ]:
#Merge the models
open3SPN2.writePDB(model,'clean.pdb')

In [ ]:
#Save the protein_sequence
ffAWSEM.save_protein_sequence(model,sequence_file='protein.seq')

In [ ]:
#Fix memories (replace in single_frags.mem)
with open('single_frags.mem','w+') as single_frags_mem_file:
    single_frags_mem_file.write('''[Target]
query

[Memories]
''')
    for chain in model['chainID'].unique()[2:-1]:
        serial = model[model['chainID']==chain].iloc[0]['serial']
        l = len(model[model['chainID']==chain]['resSeq'].unique())
        if l==274:
            single_frags_mem_file.write(f'fixed_E.gro {serial} 1 {l} 20\n')
        else:
            single_frags_mem_file.write(f'fixed_F.gro {serial} 1 {l} 20\n')

# Molecular Dynamics

In [ ]:
# Import openAWSEM, open3SPN2 and other libraries
import open3SPN2
import ffAWSEM

import pandas
import numpy as np
import simtk.openmm

from functools import partial
import sys

In [ ]:
#Create the merged system
pdb=simtk.openmm.app.PDBFile('clean.pdb')

top=pdb.topology
coord=pdb.positions
forcefield=simtk.openmm.app.ForceField(ffAWSEM.xml,open3SPN2.xml,'motor.xml')
s=forcefield.createSystem(top)

In [ ]:
#Clear Forces from the system (optional)
keepCMMotionRemover=True
j=0
for i, f in enumerate(s.getForces()):
    if keepCMMotionRemover and i == 0 and f.__class__ == simtk.openmm.CMMotionRemover:
        # print('Kept ', f.__class__)
        j += 1
        continue
    else:
        # print('Removed ', f.__class__)
        s.removeForce(j)
if keepCMMotionRemover == False:
    assert len(s.getForces()) == 0, 'Not all the forces were removed'
else:
    assert len(s.getForces()) <= 1, 'Not all the forces were removed'

In [ ]:
# Calculate anchor indices
model=open3SPN2.parsePDB('clean.pdb')
anchors = (model[(model['name']=='S') & (model['chainID'].isin(['A']))].iloc[0].name, 
           model[(model['name']=='S') & (model['chainID'].isin(['A']))].iloc[-1].name, 
           model[(model['name']=='S') & (model['chainID'].isin(['B']))].iloc[0].name, 
           model[(model['name']=='S') & (model['chainID'].isin(['B']))].iloc[-1].name)
anchors

In [ ]:
import openmm
forces={}
for i in range(s.getNumForces()):
    force = s.getForce(i)
    force_name="CMMotionRemover"
    forces.update({force_name:force})

#Add motor forces
# Define the bond force between pairs of particles
motor_beads=model[model['resname']=='X'].index.values
bond_force = openmm.CustomCompoundBondForce(2,'k1*(distance(p1,p2)-r0)^2+k2*(z1-z2)^2')
bond_force.addPerBondParameter('k1')  # spring constant
bond_force.addPerBondParameter('k2') # plane constraint
bond_force.addPerBondParameter('r0')  # equilibrium bond length
dS = 13.491521490610054 # Distance between sugars
for i,j in motor_beads.reshape(-1,2):
    bond_force.addBond([i, j], [1000.0, 1000.0 ,dS/10])  # add a bond between particles 2i and 2i+1 with k=100 kJ/mol/nm^2 and r0=1 nm
bond_force.setForceGroup(5)

#Repulsion force and alignment
motor_pairs=np.array([a for a in zip(motor_beads.reshape(-1,2)[0:-1],motor_beads.reshape(-1,2)[1:])]).reshape(-1,4)
    
repulsion_force = openmm.CustomCompoundBondForce(4,"""k*step(cutoff-Mz)*(Mz-cutoff)^2 + k2*step(Mxy-cutoff2)*(Mxy-cutoff2)^2;
                                                      Mxy=sqrt(Mx^2+My^2);
                                                      Mx=(x4+x3-x2-x1)/2;
                                                      My=(y4+y3-y2-y1)/2;
                                                      Mz=(z4+z3-z2-z1)/2""")
repulsion_force.addPerBondParameter('k')  # spring constant
repulsion_force.addPerBondParameter('cutoff')  # spring constant
repulsion_force.addPerBondParameter('k2')  # spring constant
repulsion_force.addPerBondParameter('cutoff2')  # spring constant

for i,j,k,l in motor_pairs:
    repulsion_force.addBond([i, j, k, l], [10000.0, .3,100,5])
repulsion_force.setForceGroup(5)
    
#Add twist force
force_expression = f"""bias* (1 - cos(angle - angle_0));
                       angle=theta - 2*step(dot_sign)*(theta - {np.pi});
                       dot_sign=Ex*Mx+Ey*My+Ez*Mz;
                       theta=acos(max(-1,min(1,dot)));
                       dot=(Cx*Dx+Cy*Dy+Cz*Dz)/Cm/Dm;
                       Dm=sqrt(Dx^2+Dy^2+Dz^2)+1E-3;
                       Cm=sqrt(Cx^2+Cy^2+Cz^2)+1E-3;
                       Ex=Cy*Dz-Cz*Dy;
                       Ey=Cz*Dx-Cx*Dz;
                       Ez=Cx*Dy-Cy*Dx;
                       Dx=By*Mz-Bz*My;
                       Dy=Bz*Mx-Bx*Mz;
                       Dz=Bx*My-By*Mx;
                       Cx=Ay*Mz-Az*My;
                       Cy=Az*Mx-Ax*Mz;
                       Cz=Ax*My-Ay*Mx;
                       Mx=(x4+x3-x2-x1)/2;
                       My=(y4+y3-y2-y1)/2;
                       Mz=(z4+z3-z2-z1)/2;
                       Ax=(x2-x1);
                       Ay=(y2-y1);
                       Az=(z2-z1);
                       Bx=(x4-x3);
                       By=(y4-y3);
                       Bz=(z4-z3);"""

twist = simtk.openmm.CustomCompoundBondForce(4,force_expression)
twist.addGlobalParameter('bias',10000)
twist.addGlobalParameter('angle_0',0)

for i,j,k,l in motor_pairs:
    twist.addBond([i,j,k,l])
twist.setForceGroup(4)    

#Add anchor bonds
anchor_bond=simtk.openmm.CustomBondForce("0.5*k*r^2")
anchor_bond.addPerBondParameter("k");
for pair in zip(motor_beads[[0,-2,-1,1]],anchors):
    d=0
    anchor_bond.addBond(pair[0],pair[-1],[100.0])
anchor_bond.setForceGroup(5)    

s.addForce(bond_force)
s.addForce(repulsion_force)
s.addForce(twist)
s.addForce(anchor_bond)

forces.update({'twist':twist})
forces.update({'bond_force':bond_force})



In [ ]:
#Create the DNA and Protein Objects
dna=open3SPN2.DNA.fromCoarsePDB('clean.pdb')
with open('protein.seq') as ps:
    protein_sequence_one=ps.readlines()[0]
protein=ffAWSEM.Protein.fromCoarsePDB('clean.pdb',sequence=protein_sequence_one)
dna.periodic=False
protein.periodic=False

#Copy the AWSEM parameter files
ffAWSEM.copy_parameter_files()

In [ ]:
#Add 3SPN2 forces
for force_name in open3SPN2.forces:
    print(force_name)
    force = open3SPN2.forces[force_name](dna)
    if force_name in ['BasePair','CrossStacking']:
        force.addForce(s)
    else:
        s.addForce(force)
    forces.update({force_name:force})

#Add AWSEM forces
openAWSEMforces = dict(Connectivity=ffAWSEM.functionTerms.basicTerms.con_term,
                       Chain=ffAWSEM.functionTerms.basicTerms.chain_term,
                       Chi=ffAWSEM.functionTerms.basicTerms.chi_term,
                       Excl=ffAWSEM.functionTerms.basicTerms.excl_term_v2,
                       rama=ffAWSEM.functionTerms.basicTerms.rama_term,
                       rama_pro=ffAWSEM.functionTerms.basicTerms.rama_proline_term,
                       contact=ffAWSEM.functionTerms.contactTerms.contact_term,
                       frag  = partial(ffAWSEM.functionTerms.templateTerms.fragment_memory_term,
                                       frag_file_list_file="./single_frags.mem",
                                       npy_frag_table="./single_frags.npy",
                                       UseSavedFragTable=False,
                                       k_fm=0.04184/3),
                       beta1 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_1,
                       beta2 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_2,
                       beta3 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_3,
                       pap1 = ffAWSEM.functionTerms.hydrogenBondTerms.pap_term_1,
                       pap2 = ffAWSEM.functionTerms.hydrogenBondTerms.pap_term_2,
                      )
protein.setup_virtual_sites(s)

#Add DNA-protein interaction forces
for force_name in open3SPN2.protein_dna_forces:
    print(force_name)
    force = open3SPN2.protein_dna_forces[force_name](dna,protein)
    s.addForce(force)
    forces.update({force_name: force})

#Fix exclussions
for force_name in openAWSEMforces:
    print(force_name)
    if force_name in ['contact']:
        force = openAWSEMforces[force_name](protein, withExclusion=False,periodic=False)
        print(force.getNumExclusions())
        open3SPN2.addNonBondedExclusions(dna,force)
        print(force.getNumExclusions())
    elif force_name in ['Excl']:
        force = openAWSEMforces[force_name](protein)
        print(force.getNumExclusions())
        open3SPN2.addNonBondedExclusions(dna,force)
        print(force.getNumExclusions())
    else:
        force = openAWSEMforces[force_name](protein)
    s.addForce(force)
    forces.update({force_name: force})

In [ ]:
# Set up the simulation
temperature=300 * simtk.openmm.unit.kelvin
platform_name='OpenCL' #'Reference','CPU','CUDA', 'OpenCL'

integrator = simtk.openmm.LangevinIntegrator(temperature, 1 / simtk.openmm.unit.picosecond, 2 * simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(top,s, integrator, platform)
simulation.context.setPositions(coord)
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)

In [ ]:
#Obtain total energy

energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',round(energy,6),energy_unit.get_symbol())

#Obtain detailed energy

energies = {}
for force_name, force in forces.items():
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

In [ ]:
#Add simulation reporters
dcd_reporter=simtk.openmm.app.DCDReporter(f'output.dcd', 1000)
energy_reporter=simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,
                                                   potentialEnergy=True, temperature=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(energy_reporter)

#debug_reporter=simtk.openmm.app.DCDReporter(f'output0.dcd', 1000)
#simulation.reporters.append(debug_reporter)

In [ ]:
dna.atoms.to_csv('test.csv')

In [ ]:
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)

twist_0=0
force_names = list(forces.keys())
num_forces = len(force_names)
data_array = np.empty((5000, num_forces + 1))

for i in range(10000):
    #Run simulation
    twist_0+=1.5*np.pi/10000
    simulation.context.setParameter('angle_0',twist_0)
    
    #debug_reporter=simtk.openmm.app.DCDReporter(f'output{i%5}.dcd', 1)
    #simulation.reporters[-1]=debug_reporter    

    #for j in range(1000):
    simulation.step(100)

#         groups = list(set([2**force.getForceGroup() for force in forces.values()]))
#         all_groups = sum(groups)
#         state = simulation.context.getState(getEnergy=True)
        
#         # Shift data_array to make room for new data
#         #data_array = np.roll(data_array, shift=-1, axis=0)
        
#         # First column is total energy
#         #data_array[-1, 0] = state.getPotentialEnergy().value_in_unit(energy_unit)
        
#         # Subsequent columns are group energies
#         for k, (force_name, force) in enumerate(forces.items()):
#             group = force.getForceGroup()
#             state=simulation.context.getState(getEnergy=True, groups=2**group)
#             group_energy = state.getPotentialEnergy().value_in_unit(energy_unit)
#             #data_array[-1, k + 1] = group_energy

# Convert data to DataFrame
# columns = ['TotalEnergy'] + force_names
#data = pandas.DataFrame(data_array, columns=columns)
